In [1]:
import cyipopt
import jax
import jax.numpy as jnp
import numpy as np
from typing import *

In [321]:
from sdf_world.sparse_ipopt import *

In [330]:
#function
ws_lb = np.array([-1,-1,-0.5, -np.pi, -np.pi, -np.pi])
ws_ub = np.array([1,1,1.5, np.pi, np.pi, np.pi])

grasp_fn = lambda g: 0.
manip_fn = lambda g, pose: 0.
dist_fn = lambda g1, g2, pose, pose_st, pose_ed: jnp.zeros(4)


jac_grasp_fn1 = lambda g: jnp.zeros(3)
jac_manip_fn1 = lambda g, pose: jnp.zeros(3)
jac_manip_fn2 = lambda g, pose: jnp.zeros(6)
jac_dist_fn1 = lambda g1, g2, pose, pose_st, pose_ed: jnp.zeros((4,3))
jac_dist_fn2 = lambda g1, g2, pose, pose_st, pose_ed: jnp.zeros((4,3))
jac_dist_fn3 = lambda g1, g2, pose, pose_st, pose_ed: jnp.zeros((4,6))
jac_grasp_fns = [jac_grasp_fn1]
jac_manip_fns = [jac_manip_fn1, jac_manip_fn2]
jac_dist_fns = [jac_dist_fn1, jac_dist_fn2, jac_dist_fn3, None, None]


builder = SparseIPOPTBuilder()

builder.add_variable("g_pick", 3, -1., 1.)
builder.add_variable("g_place", 3, -1., 1.)
builder.add_variable("p_ho", 6, ws_lb, ws_ub)
builder.add_parameter("p_start", jnp.zeros(6))
builder.add_parameter("p_goal", jnp.zeros(6))

builder.register_fn("grasp_logit_fn", [3], 1,
                          grasp_fn, jac_grasp_fns)
builder.register_fn("manip_fn", [3, 6], 1,
                          manip_fn, jac_manip_fns)
builder.register_fn("dist_fn", [3, 3, 6, 6, 6], 4,
                          dist_fn, jac_dist_fns)
builder.register_fn("debug_obj", [], 1, 
                           lambda : 0., [])

builder.add_objective([], "debug_obj")
builder.add_constr("grasp_prob_pick", ["g_pick"], "grasp_logit_fn",
                   1., np.inf)
builder.add_constr("grasp_prob_place", ["g_place"], "grasp_logit_fn",
                   1., np.inf)
builder.add_constr("manip_pick", ["g_pick", "p_start"],"manip_fn",
                   0.3, np.inf)
builder.add_constr("manip_place", ["g_place", "p_goal"], "manip_fn",
                   0.3, np.inf)
builder.add_constr("manip_ho_1", ["g_pick", "p_ho"], "manip_fn",
                   0.3, np.inf)
builder.add_constr("manip_ho_2", ["g_place", "p_ho"], "manip_fn",
                   0.3, np.inf)
builder.add_constr("dist", ["g_pick", "g_place", "p_ho", "p_start", "p_goal"], "dist_fn",
                   0.05, np.inf)
builder.freeze()

In [331]:
builder.get_objective_fn()(jnp.zeros(builder.xdim))

0.0

In [332]:
builder.get_gradient_fn()(jnp.zeros(builder.xdim))

Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [320]:
ipopt = builder.build(True)


Sparsity pattern:
00 01 02 -- -- -- -- -- -- -- -- -- 
-- -- -- 03 04 05 -- -- -- -- -- -- 
06 07 08 -- -- -- -- -- -- -- -- -- 
-- -- -- 09 10 11 -- -- -- -- -- -- 
12 13 14 -- -- -- 18 19 20 21 22 23 
-- -- -- 15 16 17 24 25 26 27 28 29 
30 31 32 42 43 44 54 55 56 57 58 59 
33 34 35 45 46 47 60 61 62 63 64 65 
36 37 38 48 49 50 66 67 68 69 70 71 
39 40 41 51 52 53 72 73 74 75 76 77 
